In [60]:
import platform as python_platform

class Platform:
    WINDOWS = "Windows"
    MACOS = "Darwin"

platform = python_platform.system()

windows_root_directory = "D:/Vector A/0. KHTN/Nam 4/HKII/Thesis/Brainstorming/DataCrawling"
mac_root_directory = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/DataCrawling"

root_directory = windows_root_directory if platform == Platform.WINDOWS else mac_root_directory

In [61]:
import os
from os.path import join, getsize

import sys
sys.path.append(root_directory)

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
# from requests_html import HTMLSession

from datetime import date, timedelta
import datetime
from datetime import timezone
import time

import pandas as pd

from pathlib import Path
import os
import json
import threading

import file_utils as fu

In [62]:
class ResponseType:
    JSON = "json"
    XML = "xml"

In [63]:
def copy_folder_structure(source, destination):
    '''Copy the structure of the source directory to the destination directory'''
    for root, dirs, files in os.walk(source):
        for dir in dirs:
            new_dir = f"{destination}/{root.replace(source, '')}/{dir}"
            
            regex_pattern = r"\/{2,}"            
            new_dir = re.sub(regex_pattern, "/", new_dir)

            os.makedirs(new_dir, exist_ok=True)

In [64]:
song_ids_of_song_parts_dir = fu.relative_to_absolute_path(
    "DataPreprocessing/song_id_of_song_parts",
    root_path=root_directory
)

song_parts_from_ids_dir = fu.relative_to_absolute_path(
    "DataPreprocessing/song_parts_from_ids",
    root_path=root_directory
)

copy_folder_structure(
    source=song_ids_of_song_parts_dir,
    destination=song_parts_from_ids_dir
)

In [65]:
def make_request(url, dict_parameters={}, dict_headers={}):
    session = Session()
    session.headers.update(dict_headers)
    
    status_code = None
    data_type = None
    data = None
    
    try:
        response = session.get(url, params=dict_parameters)
        
        # Kiểm tra response trả về ở dạng json hay xml
        if response.headers['Content-Type'] == 'application/json; charset=utf-8':
            data_type = ResponseType.JSON
            data = response.json()
        else:
            data_type = ResponseType.XML
            data = response.text
        
        status_code = response.status_code
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        status_code = response.status_code
        print(e)

    return {
        'status_code' : status_code,
        'data_type' : data_type,
        'data' : data
    }

In [66]:
# https://api.hooktheory.com/v1/songs/public/jDgXpWlLgKl
result = make_request("https://api.hooktheory.com/v1/songs/public/ZwxKJrPGged")
print(result)


{'status_code': 200, 'data_type': 'xml', 'data': '{"ID":133315,"user_id":103696,"dateModified":"2018-07-31 12:44:48+00:00","HTID":"ogmC3LVRvUiD","artist":"Armin van Buuren","song":"This Is What It Feels Like","section":"Chorus","revision":1,"chords":null,"youTubeID":"MS1a-SQd79w","xmlData":"<super>\\n  <meta>\\n    <artist>Armin van Buuren</artist>\\n    <title>This Is What It Feels Like</title>\\n    <beats_in_measure>4</beats_in_measure>\\n    <BPM>128</BPM>\\n    <key>F</key>\\n    <YouTubeID>MS1a-SQd79w</YouTubeID>\\n    <mode>6</mode>\\n    <sections>\\n      <_Chorus>\\n        <global_start>77.39</global_start>\\n        <section_duration>29.55</section_duration>\\n        <active_start>1.84</active_start>\\n        <active_stop>27.7</active_stop>\\n      </_Chorus>\\n    </sections>\\n  </meta>\\n  <sections>\\n    <_Chorus>\\n      <segment>\\n        <instrument/>\\n        <rhythm/>\\n        <lyrics/>\\n        <notes>\\n          <note>\\n            <start_beat_abs>0</sta

In [68]:
test_file_path = '/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/DataCrawling/DataPreprocessing/song_parts_from_ids/test.json'
# Lưu data vào file json
with open(test_file_path, 'w') as file:
    file.write(result['data'])

print(test_file_path)


/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/DataCrawling/DataPreprocessing/song_parts_from_ids/test.json
